<a href="https://colab.research.google.com/github/CIS-522/course-content/blob/main/tutorials/W04_Optimization/W4_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CIS-522 Week 4 Homework


**Instructor:** Lyle Ungar

**Content Creator:** Rongguang Wang, Jordan Lei

**Content Reviewer:** Ann-Katrin Reuel, Kausik Sivakumar

---
## Preface
Since this week's homework requires coding, we recommend saving this notebook in your google Drive (`File -> Save a copy in Drive`), and share the link to the final version in the submission Airtable form. 

In [9]:
#@markdown What is your Pennkey and pod? (text, not numbers, e.g. bfranklin)
my_pennkey = '' #@param {type:"string"}
my_pod = 'Select' #@param ['Select', 'upain', 'ah-damn-optimizer', 'backpropagandists', 'backpropers','excel-erators','GAN-gsters','han-not-solo','hufflefluffs','lets-taco-bout-it','natural-networkers','pytorture','sigmoids','strong-signals','the-denominators','the-travellers', 'the-weekenders', 'tomorrows-incredibles', 'brute-force']
my_email = '' #@param {type:"string"}
tutorial = 'W4D1'


# start timing
import time
try:t0;
except NameError: t0 = time.time()


# Setup

In [2]:
# imports
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset
import time
import math
import numpy as np
from bisect import bisect_right
import matplotlib.pyplot as plt
from collections import defaultdict

In [3]:
# @title Figure Settings
import ipywidgets as widgets
%matplotlib inline 
fig_w, fig_h = (8, 6)
plt.rcParams.update({'figure.figsize': (fig_w, fig_h)})
%config InlineBackend.figure_format = 'retina'
SMALL_SIZE = 12

plt.rcParams.update(plt.rcParamsDefault)
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=SMALL_SIZE)  # fontsize of the figure title

# Part 1: Paper Reading

Please read the paper [ADAM: A METHOD FOR STOCHASTIC OPTIMIZATION](https://arxiv.org/pdf/1412.6980.pdf) and answer the following questions. 
Note that the paper is in some ways beyond the scope of this course. You can skip most of the details (and all of the proofs) in the paper. Just look at it enough to answer the following questions.

In [10]:
#@markdown The paper says that “Some of Adam’s advantages are that, 1) its stepsizes are approximately bounded by the stepsize hyperparameter, 2) it does not require a stationary objective, 3)it works with sparse gradients, and 4) it naturally performs a form of step size annealing“. Please rephrase each of these claims,  to make sure you understand what they mean.
rephrase_advantages = '' #@param {type:"string"}

In [11]:
#@markdown In the typical case, what is the bound on the update to the weights, \theta? (Hint: it’s just a simple hyperparameter)
bound = '' #@param {type:"string"}

In [12]:
#@markdown What is the “initialization bias correction” and why is it needed?
bias_correction = '' #@param {type:"string"}

In [13]:
#@markdown In plain English,  what is the “regret” in equation 5, and how does the regret scale with the number of stochastic gradient descent steps?
regret = '' #@param {type:"string"}

# Part 2: Cyclic Cosine Annealing

We have introduced many different variants of gradient descent for global minimum search such as Momentum, Adagrad, RMSProp, and Adam. These optimizers all follow their own pattern of finding this minimum loss by giving rules on how to adjust their step size, but what if we get stuck in a local minimum of a loss landscape? Even though some optimizers try and use ideas of momentum to have the ability to swing out of a local minimum, they are not always that successful.

<p align="center">
  <img width="500" height="200" src="https://miro.medium.com/max/1400/1*9Fca3kpx3pVW8SaYz2pjpw.png">
</p>

For example, the model in the left figure slowly descends into a local minimum, while the model in the right figure jumps in and out of several local minimums, searching for a more stable one.

One possible solution is cyclic cosine annealing. Instead of trying to add various forms of learning rate decay, let’s reset our learning rate every fixed iterations so that we might be able to more easily pop out of a local minimum. Specifically, we simulate a restart of SGD once $T_i$ epochs are performed, where $i$ is the index of the run. The learning rate update rule within the $i$-th run is:
$$\eta_t=\eta^i_{min}+\frac{1}{2}(\eta^i_{max}-\eta^i_{min})(1+\cos(\frac{T_{cur}}{T_i}\pi))$$
where
*   $\eta^i_{min}$ and $\eta^i_{max}$ are the range of the learning rate;
*   $T_{cur}$ is the count of epoch since last restart;
*   $T_i$ is the width between the two restarts.

<p align="center">
  <img width="300" height="200" src="https://miro.medium.com/max/1400/1*GjdEoZcEcQF19JS1y8nVYw.png">
</p>

The above figure shows an example of cyclic consine annealing. For more details, check out this [paper](https://arxiv.org/pdf/1608.03983.pdf).

You will implement a cyclic cosine annealing learning rate schedueler to perform digit classification. Towards this goal, you have to perform the following steps:

1. Define a cyclic consine annealing function which restarts the learning rate at two milestones (5 and 10 in this case). Insert your code in the cell below.

   Hint: Modify the learning rate using below code.
   ```python
    for param_group in optimizer.param_groups: 
        param_group[’lr’] = lr
   ```

In [ ]:
def cyclic_cosine_annealing(optimizer, milestones=[5,10], eta_min=1e-6,eta_max = 1e-2, last_epoch=0):
    ####################################################################
    # Fill in missing code below (...),
    # then remove or comment the line below to test your function
    # raise NotImplementedError("Add the missing code")
    ####################################################################  
    

    
    ####################################################################  

2. Check the learning rate computed by the scheduler you just implemented using the code below. Make sure that you observe behavior similar to that in the above figure. 

In [ ]:
model = torch.nn.Linear(2, 1)
optimizer = optim.SGD(model.parameters(), lr=1e-2)

lr_list = []
for i in range(20):
    cyclic_cosine_annealing(optimizer, last_epoch=i)
    lr_list.append(optimizer.param_groups[0]['lr'])
    optimizer.step()

with plt.xkcd():
    plt.plot(range(20), lr_list, label='Cyclic Consine')
    plt.ylabel('Learning Rate')
    plt.xlabel('Epoch')
    plt.legend()
    plt.show()

3. Train a digit classifier on MNIST using your cyclic cosine annealing scheduler. We provide the training code below.

In [ ]:
# @title MNIST Classifier (run me)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    avg_loss = 0.
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        avg_loss += loss.item()
        loss.backward()
        optimizer.step()
        if batch_idx % args['log_interval'] == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    avg_loss /= len(train_loader.dataset)
    return avg_loss

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return 1. - correct / len(test_loader.dataset)

def main(args):
    # gpu configuration
    use_cuda = not args['no_cuda'] and torch.cuda.is_available()
    torch.manual_seed(args['seed'])
    device = torch.device('cuda' if use_cuda else 'cpu')
    # data loader configuration
    train_kwargs = {'batch_size': args['batch_size']}
    test_kwargs = {'batch_size': args['test_batch_size']}
    if use_cuda:
        cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
        train_kwargs.update(cuda_kwargs)
        test_kwargs.update(cuda_kwargs)
    # load MNIST dataset
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
    train_loader = torch.utils.data.DataLoader(datasets.MNIST('../data', train=True, download=True,
                       transform=transform),**train_kwargs)
    test_loader = torch.utils.data.DataLoader(datasets.MNIST('../data', train=False,
                       transform=transform), **test_kwargs)
    # init models
    model = Net().to(device)
    # config optimizer
    optimizer = optim.SGD(model.parameters(), lr=args['lr'], momentum=args['momentum'])
    # training and testing
    train_list, test_list = [], []
    for epoch in range(1, args['epochs'] + 1):
        if args['scheduler']:
            cyclic_cosine_annealing(optimizer, last_epoch=epoch-1)
        loss = train(args, model, device, train_loader, optimizer, epoch) # train
        train_list.append(loss)
        error = test(model, device, test_loader) # test
        test_list.append(error)

    return train_list, test_list

4. Run the below code to train the network.  Compare the training and testing error of SGD and SGD with cyclic cosine annealing learning rate.

In [ ]:
# @title Train and Plot (run me)
# Training settings
args = {'batch_size': 64,
        'test_batch_size': 1000,
        'epochs': 20,
        'lr': 0.01,
        'scheduler': False,
        'momentum': 0.9,
        'net_type': 'Net',
        'no_cuda': False,
        'seed': 1,
        'log_interval': 100
        }

lr_anneal = [False, True]
loss_dict = {}

for i in range(len(lr_anneal)):
    args['scheduler'] = lr_anneal[i]
    train_loss, test_error = main(args)
    loss_dict['train' + str(lr_anneal[i])] = train_loss
    loss_dict['test' + str(lr_anneal[i])] = test_error

with plt.xkcd():
    fig, axs = plt.subplots(1, 2, figsize=(10,4))
    axs[0].plot(loss_dict['trainFalse'], label='SGD', color='b')
    axs[1].plot(loss_dict['testFalse'], label='SGD', color='b', linestyle='dashed')
    axs[0].plot(loss_dict['trainTrue'], label='Cyclic Cosine', color='r')
    axs[1].plot(loss_dict['testTrue'], label='Cyclic Cosine', color='r', linestyle='dashed')
    axs[0].set_title('Train')
    axs[1].set_title('Test')
    axs[0].set_ylabel('Loss')
    axs[1].set_ylabel('Error')
    axs[0].set_xlabel('Epoch')
    axs[1].set_xlabel('Epoch')
    axs[0].legend()
    axs[1].legend()
    plt.tight_layout()
    plt.show()

# Part 3: Stochastic Weight Averaging (SWA) Optimizer

In this section, you'll implement a new optimizer to perform digit classification in the MNIST dataset.

<p align="center">
  <img width="800" height="200" src="https://miro.medium.com/max/2000/1*_USiR_z8PKaDuIcAs9xomw.png">
</p>

The intuition for SWA comes from the empirical observation that local minima tend to accumulate at the border of areas on loss surface where the loss value is low (points $W_1$, $W_2$ and $W_3$ are at the border of the red area of low loss in the left panel of figure above). By taking the average of several such points, it is possible to achieve a wide, generalizable solution with even lower loss ($W_{SWA}$ in the left panel of the figure above).

Here is how it works. Instead of an ensemble of many models, you only need two models:

$$w_{SWA}\leftarrow \frac{w_{SWA}\cdot n_{models}+w}{n_{models}+1}$$

*   The first model stores the running average of model weights ($w_{SWA}$ in the above formula). This will be the final model after the end of the training, and will be used for predictions.
*   The second model ($w$ in the formula) will be traversing the weight space.

At the end of each training epoch, the current weights of the second model will be used to update the weight of the running average model by taking the weighted mean between the old running average weights and the new set of weights from the second model.(See formula above.) By following this approach, we only need to train one model, and store only two models in memory during training. For prediction, we only need the running average model.

Here is the pseudo code of the algorithm:
  ```python
    # Algorithm: Stochastic Weight Averaging

    Require: Weight w_hat, learning rate lr, epochs n
    Initialize with w <- w_hat and w_swa <- w
    for i = 1, 2,..., n do
        w <- w - lr * d(L_i)/dw
        n_models <- i
        w_swa <- (w_swa * n_models + w) / (n_models + 1)
    end for
    Output: w_swa
  ```
<p align="center">
  <img width="450" height="250" src="https://pytorch.org/assets/images/swapytorch4.png">
</p>

We expect solutions that are centered in the flat region of the loss to generalize better than those near the boundary. Indeed, train and test error surfaces are not perfectly aligned in the weight space. Solutions that are centered in the flat region are not as susceptible to the shifts between train and test error surfaces as those near the boundary. In the figure above, the train and test error surfaces along the direction connecting the SWA and SGD solutions. As you can see, while the SWA solution has a higher training loss than the SGD solution, it is centered in a region of low loss and has a substantially better test error. To see more details about SWA, check the [paper](https://arxiv.org/pdf/1803.05407.pdf).
  

You will implement SWA to perform digit classification. Towards this goal, you have to perform the following steps:

1. Define a neural network to digit classification (provided).

In [ ]:
# @title Network Class (run me)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

2. Implement training and testing procedures (provided).

In [ ]:
# @title Train and Test functions (run me)
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    avg_loss = 0.
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        avg_loss += loss.item()
        loss.backward()
        optimizer.step()
        if batch_idx % args['log_interval'] == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    avg_loss /= len(train_loader.dataset)
    return avg_loss

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return 1. - correct / len(test_loader.dataset)

3. Define the SWA moving average function. Insert your code in the cell below.

In [ ]:
def moving_average(net1, net2, swa_n):
    for param1, param2 in zip(net1.parameters(), net2.parameters()):
        ####################################################################
        # Fill in missing code below (...),
        # then remove or comment the line below to test your function
        #Read the funciton call below and build appropriately
        # raise NotImplementedError("Add the missing code")
        ####################################################################  
        

        
        ####################################################################  

4. Load the MNIST dataset from torchvision, initialize models, optimizers, and learning rate schedueler (provided).

5. Finish the SWA update rule. Insert your code in the cell below.

In [ ]:
def main(args):
    # gpu configuration
    use_cuda = not args['no_cuda'] and torch.cuda.is_available()
    torch.manual_seed(args['seed'])
    device = torch.device('cuda' if use_cuda else 'cpu')
    # data loader configuration
    train_kwargs = {'batch_size': args['batch_size']}
    test_kwargs = {'batch_size': args['test_batch_size']}
    if use_cuda:
        cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
        train_kwargs.update(cuda_kwargs)
        test_kwargs.update(cuda_kwargs)
    # load MNIST dataset
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
    train_loader = torch.utils.data.DataLoader(datasets.MNIST('../data', train=True, download=True,
                       transform=transform),**train_kwargs)
    test_loader = torch.utils.data.DataLoader(datasets.MNIST('../data', train=False,
                       transform=transform), **test_kwargs)
    # init models
    model = Net().to(device)
    swa_model = Net().to(device) # new introduced model!
    swa_n = 0
    # config optimizer and learnng rate scheduler
    optimizer = optim.SGD(model.parameters(), lr=args['lr'], momentum=args['momentum'])
    scheduler = StepLR(optimizer, step_size=1, gamma=args['gamma'])
    # training and testing
    train_list, test_list, swa_list = [], [], []
    for epoch in range(1, args['epochs'] + 1):
        loss = train(args, model, device, train_loader, optimizer, epoch) # train
        train_list.append(loss)
        error = test(model, device, test_loader) # test
        test_list.append(error)
        # swa update rule
        if epoch >= args['swa_start']:
            ####################################################################
            # Fill in missing code below (...),
            # then remove or comment the line below to test your function
            raise NotImplementedError("Add the missing code")
            ####################################################################  
            

            
            ####################################################################
        swa_error = test(swa_model, device, test_loader) # test swa
        swa_list.append(swa_error)
        scheduler.step()

    return train_list, test_list, swa_list

6. Starting training and testing. Then plot the training loss and testing error comparison between SGD and SWA using the code provided.
  **Note**: we might not be able to observe an expected performance of SWA, since MNIST calssification is relatively a simple task compared to CIFAR-10 / CIFAR-100. 

In [ ]:
# @title Train and Plot (run me)
# Training settings
args = {'batch_size': 64,
        'test_batch_size': 1000,
        'epochs': 20,
        'swa_start': 10,
        'lr': 0.01,
        'momentum': 0.9,
        'gamma': 0.7,
        'net_type': 'Net',
        'no_cuda': False,
        'seed': 1,
        'log_interval': 100
        }

train_loss, test_error, swa_error = main(args)

with plt.xkcd():
    plt.plot(train_loss, label='Train Loss', linewidth=2)
    plt.plot(test_error, label='SGD Test Error', linewidth=2)
    plt.plot(range(10, 20), swa_error[10:], label='SWA Test Error', linewidth=2)
    plt.xlabel('Epoch')
    plt.legend()
    plt.show()

# Part 4: Better-Know-Your-Pod

**List 2 members of your pod you haven't written about and write a thoughtful and nice short blurb about them (around 100 words for each pod member). Where are they from? What do they study? What are their hobbies? What do you have in common?**



In [14]:
better_know_your_pod = '' #@param {type:"string"}

# Submission
Double check that the answers have been correctly formatted in the Airtable below, and edit them as desired. In case the form below isn't working, you can submit your answers manually here: https://airtable.com/shrpDsn0Tem8iXFNX

In [ ]:
#@markdown #Run Cell to Show Airtable Form
#@markdown ##**Confirm your answers and then click "Submit"**
import numpy as np
import time
from IPython.display import IFrame
def prefill_form(src, fields: dict):
  '''
  src: the original src url to embed the form
  fields: a dictionary of field:value pairs,
  e.g. {"pennkey": my_pennkey, "location": my_location}
  '''
  prefills = "&".join(["prefill_%s=%s"%(key, fields[key]) for key in fields])
  src = src + prefills
  src = "+".join(src.split(" "))
  return src

#autofill fields if they are not present
#a missing pennkey and pod will result in an Airtable warning
#which is easily fixed user-side.
try: my_pennkey;
except NameError: my_pennkey = ""
try: my_pod;
except NameError: my_pod = "Select"
try: my_email;
except NameError: my_email = ""
try: rephrase_advantages;
except NameError: rephrase_1 = ""
try: bound;
except NameError: bound = ""
try: bias_correction;
except NameError: bias_correction = ""
try: regret;
except NameError: regret = ""
try: better_know_your_pod;
except NameError: better_know_your_pod = ""


#autofill time if it is not present
try: t0;
except NameError: t0 = time.time()
try: t1;
except NameError: t1 = time.time()
times = np.array([t1])-t0

fields = {"pennkey": my_pennkey,
          "pod": my_pod,
          "email": my_email,
          "rephrase_advantages":rephrase_advantages,
          "bound":bound,
          "bias_correction":bias_correction,
          "regret":regret,
          "better_know_your_pod": better_know_your_pod,
          "cumulative_times": times}

src = "https://airtable.com/embed/shrpDsn0Tem8iXFNX?"

#now instead of the original source url, we do: src = prefill_form(src, fields)
display(IFrame(src = prefill_form(src, fields), width = 800, height = 400))